In [2]:
import numpy as np
import re
import requests
import pandas as pd
from requests.exceptions import RequestException
import matplotlib.pyplot as plt
from datetime import datetime
import shelve
import os

class movie():  # 定义movie类

    # 电影Information
    name = None
    rating = None
    director = None
    premiereDate = None
    Duration = None
    # 人数相关
    HaveWatched_num = None
    rating_num = None
    WantWatch_num = None
    # 我的相关
    myRate = None
    my_watch_time = None
    my_comment = None
    episode = None

headers ={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
          AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 \
              Safari/537.36 OPR/66.0.3515.115'}

def get_page(url):
    try:
        resp = requests.get(url,headers=headers)
        if resp.status_code == 200:
            return resp.text
        return None
    except RequestException:
        return None

In [2]:
url_origin = 'https://movie.douban.com/people/inoruu/collect?start=0&sort=time&rating=all&filter=all&mode=grid'
html_origin = get_page(url_origin)

if html_origin == None:
    print('get the html of origin failed')
else:
    re_of_max_page = r'看过的影视\((\d+)\)'
    MovieNum = int(re.findall(re_of_max_page,html_origin)[0])
    MaxPage = MovieNum//15 + 1
    print(f'看过的电影数量为{MovieNum}, 需要爬取的页面数量为{MaxPage}')

看过的电影数量为843, 需要爬取的页面数量为57


In [ ]:
#  to get htmls of all page in movie.douban (number is Maxpage)
urls = list(range(MaxPage)) #页面最大数量
htmls = list(range(MaxPage))

for i in range(MaxPage): #页面最大数量
    urls[i] = 'https://movie.douban.com/people/inoruu/collect?start={}&sort=time&rating=all&filter=all&mode=grid'.format(i*15)
    htmls[i] = get_page(urls[i])
    if htmls[i] != None:
        print('get the htmls of page{}'.format(i+1))
    else:
        print('get the htmls of page{} failed'.format(i+1))
    
print(f'get all {MaxPage} htmls')

url_of_single_movie = []
re_of_url = r'<li class="title">.*?<a href="(.*?)".*?'
for i in range(MaxPage):
    temp = re.findall(re_of_url,htmls[i],re.S)
    for j in range(len(temp)):
        url_of_single_movie.append(temp[j])

In [ ]:
# 保存html时使用这个cell
HTMLshelf = shelve.open('HTMLshelf'+datetime.today().strftime('%Y-%m-%d')) 
HTMLshelf['my_movie_list'] = htmls # 保存htmls
HTMLshelf['url_of_single_movie'] = url_of_single_movie # 保存url

In [3]:
# 打开html时使用这个cell
os.chdir('D:\\code\\douban\\shelve')
HTMLshelf = shelve.open('HTMLshelf'+'2024-08-24') 
htmls = HTMLshelf['my_movie_list']

In [ ]:
# 提取单个电影html，输出无法爬取的url
movienames = []
re_of_name = r'<li class="title">.*?<em>(.*?)</em>'
re_Maintext = r'<div class="item comment-item".*?<ul>.*?</ul>.*?</div>'

for i in range(MaxPage):
    Maintext = re.findall(re_Maintext,htmls[i],re.S)
    for j in range(len(Maintext)):
        movienames.append(re.findall(re_of_name,Maintext[j],re.S))

count = 0
badcount = 0
badmovie = []

html_all = [None] * MovieNum

for i in range(MovieNum):
   html_all[i] = get_page(url_of_single_movie[i])
   if html_all[i] == None:
      print('this subject: ' + movienames[i][0] + ' can\'t be scrapped')
      badmovie.append(movienames[i][0])
      badcount += 1
      print(f'{badcount} bad movie.')
   else:
      count += 1
      print(f'{count} has been scrapped.')

print('bad movie list: ',badmovie)

In [6]:
# 保存时用
AllHTMLs = shelve.open('html_all'+datetime.today().strftime('%Y-%m-%d')) 
AllHTMLs['html_all'] = html_all # 保存htmls
AllHTMLs.close()

In [38]:
# 打开时用
AllHTMLs = shelve.open('html_all'+'2024-08-24')
html_all = AllHTMLs['html_all']
AllHTMLs.close()


In [35]:
# 在我的电影页面，获取电影名，我的评分，评分时间
movielist = []
re_of_name = r'<li class="title">.*?<em>(.*?)</em>'
re_of_myRate = r'<span class="rating(\d)-t"'
re_of_my_watch_time = r'<li class="title".*?<span class="date">(.*?)</span>'
re_comment = r'<span class="comment">(.*?)</span>'
re_Maintext = r'<div class="item comment-item".*?<ul>.*?</ul>.*?</div>'

for i in range(len(htmls)):
    Maintext = re.findall(re_Maintext,htmls[i],re.S)
    for j in range(len(Maintext)):
        temp = movie()
        temp.name = re.findall(re_of_name,Maintext[j],re.S)
        temp.my_comment = re.findall(re_comment,Maintext[j],re.S)
        if 'rating' in Maintext[j]:
            temp.myRate = re.findall(re_of_myRate,Maintext[j],re.S)
            temp.myRate = int(temp.myRate[0])
        else:
            temp.myRate = np.nan
        if re.findall(re_comment,Maintext[j],re.DOTALL) != []:
            temp.my_comment = re.findall(re_comment,Maintext[j],re.DOTALL)[0]
        else:
            temp.my_comment = 'no comment'

        temp.my_watch_time = re.findall(re_of_my_watch_time,Maintext[j],re.S)[0]
    
        movielist.append(temp)

    # print('page {} was finished.'.format(i+1))

In [52]:
re_of_rating = r'<strong class="ll rating_num".*?(\d+\.\d+).*?</strong>'
re_of_HaveWatched_num = r'<div class="subject-others-interests-ft".*?<a href.*?(\d+).*?(\d+).*?(\d+).*?(\d+).*?</div>'
re_of_rating_num = r'<div class="rating_sum".*?(\d+).*?</div>'
re_of_WantWatch_num = r'<div class="subject-others-interests-ft".*?<a href.*?(\d+).*?(\d+).*?(\d+).*?(\d+).*?</div>'
re_of_director = r'"director".*?"name":\s*"([^"]+)"'
re_of_premiereDate = r'"datePublished":\s*"(\d{4}-\d{2}-\d{2})"'
re_of_Duration = r'"duration":\s*"PT(\d+H)?(\d+M)?"'
re_episode = r'集数:</span>\s*(\d+)'


In [ ]:
count = 0
for i in range(len(html_all)):
    if i==135:
        count += 1
        continue

    if html_all[i] != None:
        movielist[i].rating = float(re.findall(re_of_rating,html_all[i],re.S)[0])
        movielist[i].HaveWatched_num = int(re.findall(re_of_HaveWatched_num,html_all[i],re.S)[0][1])
        movielist[i].rating_num = int(re.findall(re_of_rating_num,html_all[i],re.S)[0])
        movielist[i].WantWatch_num = int(re.findall(re_of_WantWatch_num,html_all[i],re.S)[0][3]) 
        if re.findall(re_of_premiereDate,html_all[i],re.S) != []:
            movielist[i].premiereDate = re.findall(re_of_premiereDate,html_all[i],re.S)[0]
        if re.findall(re_of_director,html_all[i],re.S) != []:
            movielist[i].director = re.findall(re_of_director,html_all[i],re.S)[0]
        if re.findall(re_of_Duration,html_all[i],re.S) != []:
            match = re.search(re_of_Duration,html_all[i])
            if match:
                hours = match.group(1)
                minutes = match.group(2)
                total_minutes = 0
                if hours:
                    total_minutes += int(hours[:-1]) * 60  # 去掉H并转换为分钟

                if minutes:
                    total_minutes += int(minutes[:-1])
        
            if re.findall(re_episode,html_all[i],re.S) != []:
                movielist[i].episode = int(re.findall(re_episode,html_all[i],re.S)[0])
                total_minutes = total_minutes * movielist[i].episode

            movielist[i].Duration = total_minutes
        else:
            movielist[i].Duration = np.nan
    
    count += 1
    if count%100 == 0:
        print(f'{count} has been scrapped.')
    
print(f'{count} has been scrapped.')

In [59]:
import numpy as np

attrs = ['name', 'director', 'premiereDate', 'rating', 'Duration', 'myRate', 'my_comment', 'my_watch_time', 'HaveWatched_num', 'WantWatch_num', 'rating_num']

sheet = np.full((len(movielist)+1,len(attrs)+1), None)

sheet[0,0] = 'Index'
sheet[0,1:] = attrs

for i in range(len(movielist)):
    sheet[i+1,0] = i+1
    sheet[i+1,1:] = [getattr(movielist[i], attr) for attr in attrs]

for i in range(len(sheet)):
    keyword = 'rating'
    loc = np.where(sheet[0,:] == keyword)[0]
    if sheet[i,loc[0]] == 10:
        sheet[i,loc[0]] = None


In [60]:
import pandas as pd
df = pd.DataFrame(sheet)
df.to_excel(datetime.today().strftime('%Y-%m-%d')+'movies.xlsx', index=False, header=False)

from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.utils import get_column_letter

wb = load_workbook(datetime.today().strftime('%Y-%m-%d')+'movies.xlsx')
ws = wb.active

# 设置列宽
for col in ws.columns:
    max_length = 0
    column = col[0].column_letter
    for cell in col:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(str(cell.value))
        except:
            pass
    adjusted_width = (max_length + 2)
    ws.column_dimensions[column].width = adjusted_width

# 设置行高
for row in ws.rows:
    ws.row_dimensions[row[0].row].height = 20
    for cell in row:
        if isinstance(cell.value, str) and cell.value.isdigit():
            cell.value = int(cell.value)  # 转换为数字格式

# 设置标题行格式
header_font = Font(name='Arial', size=12, bold=True)
header_fill = PatternFill(start_color='CCCCCC', end_color='CCCCCC', fill_type='solid')
for cell in ws[1]:
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = Alignment(horizontal='center', vertical='center')

# 设置数据行格式
data_font = Font(name='Arial', size=11)
for row in ws.iter_rows(min_row=2):
    for cell in row:
        cell.font = data_font
        cell.alignment = Alignment(horizontal='left', vertical='center')

# 保存修改后的Excel文件
wb.save(datetime.today().strftime('%Y-%m-%d')+'movies.xlsx')

In [23]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import numpy as np
from matplotlib.ticker import FuncFormatter
from datetime import datetime, timedelta
from scipy import interpolate
import matplotlib.font_manager as fm

try:
    plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']  # 微软雅黑
except:
    plt.rcParams['font.sans-serif'] = ['SimHei']  # 黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

sheet = pd.read_excel('2024-08-25movies.xlsx')
watch_times = sheet['my_watch_time'].tolist()
watch_times = watch_times[::-1]
view_counts = sheet['Index'].tolist()


In [24]:
watch_times = [datetime.strptime(date, '%Y-%m-%d') for date in watch_times]

# 设置起始日期（第一个观影日期前15天）
start_date = watch_times[0] - timedelta(days=15)

# 创建插值函数
time_nums = [(t - start_date).days for t in watch_times]
f = interpolate.interp1d(time_nums, view_counts, kind='linear', fill_value='extrapolate')

# 创建平滑的时间序列
smooth_times = [start_date + timedelta(days=i) for i in range((watch_times[-1] - start_date).days + 1)]
smooth_counts = f([(t - start_date).days for t in smooth_times])

# 创建图形和子图
fig, ax = plt.subplots(figsize=(12, 6))

# 初始化折线图
line, = ax.plot([], [], lw=2)

# 设置标题和标签
ax.set_title('观影数量随时间变化', fontsize=16)
ax.set_xlabel('时间', fontsize=12)
ax.set_ylabel('观影数量', fontsize=12)

# 格式化y轴标签
def format_func(value, tick_number):
    return f'{int(value):,}'

ax.yaxis.set_major_formatter(FuncFormatter(format_func))

# 动画更新函数
def update(frame):
    current_time = smooth_times[frame]
    current_count = smooth_counts[frame]
    
    x = smooth_times[:frame+1]
    y = smooth_counts[:frame+1]
    
    line.set_data(x, y)
    
    # 动态更新x轴范围，保持起始日期不变
    ax.set_xlim(start_date, current_time + timedelta(days=30))
    
    # 动态更新y轴范围
    y_max = max(y)
    ax.set_ylim(0, y_max * 1.1)
    
    # 更新标题
    ax.set_title(f'观影数量随时间变化 (截至 {current_time.strftime("%Y-%m-%d")})', fontsize=16)
    
    return line,

# 创建动画
ani = animation.FuncAnimation(fig, update, frames=len(smooth_times), interval=20, blit=False)

# 保存为MP4视频
ani.save('dynamic_view_counts_smooth.mp4', writer='ffmpeg', fps=60)

plt.close(fig)

C:\Users\99713\AppData\Local\Temp\ipykernel_16724\1450869137.py:46: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(0, y_max * 1.1)
